In [ ]:
import numpy as np
from numpy import pi
from matplotlib import pyplot as plt
import time
%matplotlib inline
from sklearn.decomposition import PCA

import pyqg
from pyqg import diagnostic_tools as tools

## setup run

In [ ]:
# domain parameters
L =  1000.e3     # length scale of box    [m]
# Ld = 15.e3       # deformation scale      [m]
# kd = 1./Ld       # deformation wavenumber [m^-1]
Nx = 64          # number of grid points
Nz = 10          # number of vertical grid points

# layer thickness and depth
H = np.array([100.,100.,100.,200.,200.,300.,400.,500.,500.,600.])
depth = np.sum(H)
z_layer = np.append(0,np.cumsum(-H))
z_full = np.arange(0,z_layer[-1],-1)

# zonal background velocity
# including unstable background velocity will permit turbulence
# U = np.zeros_like(H)
Umax = 0.0
Uslope = Umax/depth
U_func = lambda z: Umax + Uslope*z
U = np.zeros_like(H)
for ii in range(Nz):
    U[ii] = U_func( np.arange(z_layer[ii],z_layer[ii+1],-1) ).mean()

# latitude
lat = 30.        # latitude
f0 = 2*7.2921E-5*np.sin(np.deg2rad(lat)) # coriolis param [s^-1]
beta = 2*7.292e-5*np.cos(np.deg2rad(lat))/6370e3 # planetary vorticity gradient [m^-1 s^-1]
g = 9.81

# stratification
# Jeffrey has some nice examples in InternalModes.m/StratificationProfileWithName
H_strat = 1.0e3;        # thermocline exponential scale, meters
N0 = 5.2e-3;            # reference buoyancy frequency, radians/seconds
rho0 = 1025;            # mean density
rho_func = lambda z: rho0*(1 + H_strat*N0*N0/(2*g)*(1 - np.exp(2*z/H_strat)))
N2_func = lambda z: N0*N0*np.exp(2*z/H_strat)
rho = np.zeros_like(H)
for ii in range(Nz):
    rho[ii] = rho_func( np.arange(z_layer[ii],z_layer[ii+1],-1) ).mean()

# bottom topography
# used cos (not sin) so that "mode 0" topography contributes a 1, not a 0.
amp = 50
xmode = 0
ymode = 10
X,Y = np.meshgrid(
            np.arange(0.5,Nx,1.)/Nx*L,
            np.arange(0.5,Nx,1.)/Nx*L )
hbot = amp* np.cos(2*np.pi*xmode*X/L) * np.cos(2*np.pi*ymode*Y/L)

# friction parameters
rbg = 0
rek = 1.e-7       # linear bottom drag coeff.  [s^-1]

# intial PV anomaly
# sig = 1.e-7
# qi = sig*np.random.randn(Nz,Nx,Nx)
sig = 1.0e-6    # initial amplitude
kmode = 3.      # initial zonal wavelength [cycles m^-1]
qi = sig*np.sin(2*np.pi*kmode*X/L)
qi = qi[np.newaxis,:,:]*np.ones((Nz,1,1))

# time stepping parameters
# Ti = Ld/(abs(U1))  # estimate of most unstable e-folding time scale [s]
# dt = Ti/200.   # time-step [s]
# tmax = 100*Ti      # simulation time [s]
dt = 1500.        # time-step [s]
tmax = 30000000.  # simulation time [s]
twrite = 5000     # time between diagnostic checks [timesteps]
tavestart = 30*86400        # time to start averaging diagnostics [s]

In [ ]:
m = pyqg.LayeredModel(nx=Nx, nz=Nz, U=U, V=U, L=L, f=f0, beta=beta,
                         H=H, rho=rho, hbot=hbot, rek=rek, rbg=rbg,
                        dt=dt, tmax=tmax, twrite=twrite, tavestart=tavestart, ntd=1)

In [ ]:
qi = qi * m.pmodes[:,1,np.newaxis,np.newaxis]    # make initial wave baroclinic mode 1
m.set_q(qi)                # set initial PV anomaly

## run

In [ ]:
# define a quick function for plotting and visualize the initial condition
# def plot_q(m, qmax=2*qi.max()):
#     fig, ax = plt.subplots()
#     pc = ax.pcolormesh(m.x/m.rd,m.y/m.rd,m.q[0]+m.Qy[0]*m.y*0, cmap='RdBu_r')
#     pc.set_clim([-qmax, qmax])
#     plt.xlabel(r'$x/L_d$')
#     plt.ylabel(r'$y/L_d$')
#     plt.colorbar(pc)
#     plt.title('Time = %g' % m.t)
#     plt.show()
#    
# for _ in m.run_with_snapshots(tsnapstart=0, tsnapint=300*dt):
#     plot_q(m)

In [ ]:
t0 = time.time()
m.run()
t1 = time.time()
print("computation time is", t1-t0, "s")

## plot

In [ ]:
plt.figure(figsize=(14,12))

plt.subplot(221)
plt.pcolormesh(m.x/m.rd,m.y/m.rd,m.hb[-1,:,:],cmap='copper')
plt.xlabel(r'$x/L_d$')
plt.ylabel(r'$y/L_d$')
plt.colorbar()
plt.title('bathymetry [m]')

ax = plt.subplot(222)
plt.plot(np.stack((rho,rho)),np.stack((z_layer[:-1],z_layer[1:])), 'k',linewidth=2)
plt.plot(rho_func(z_full),z_full, linewidth=2)
plt.xlabel(r'$\rho\ (\mathrm{kg\,m^{-3}})$')
plt.ylabel(r'depth (m)')
plt.title('stratification profile')
ax.set_ylim(-depth,0)

plt.subplot(223)
plt.pcolormesh(m.x/m.rd,m.y/m.rd,qi[0,]+m.Qy[0]*m.y*0, cmap='RdBu_r')
plt.xlabel(r'$x/L_d$')
plt.ylabel(r'$y/L_d$')
plt.colorbar()
plt.title('initial PV')

plt.subplot(224)
plt.pcolormesh(m.y[:,0]/m.rd,z_layer[:-1],qi[:,1,:], cmap='RdBu_r')
plt.xlabel(r'$x/L_d$')
plt.ylabel(r'depth (m)')
plt.colorbar()
plt.title('initial PV')

plt.savefig("/Users/cwortham/Documents/research/rough_topo/figures/ICs.pdf", bbox_inches='tight')

In [ ]:
plt.figure(figsize=(18,4))

plt.subplot(131)
plt.pcolormesh(m.x/m.rd,m.y/m.rd,(m.q[0,]+m.Qy[0]*m.y),cmap='Spectral_r')
plt.xlabel(r'$x/L_d$')
plt.ylabel(r'$y/L_d$')
plt.colorbar()
plt.title('Layer 1 PV')

plt.subplot(132)
plt.pcolormesh(m.x/m.rd,m.y/m.rd,(m.q[1,]+m.Qy[1]*m.y),cmap='Spectral_r')
plt.xlabel(r'$x/L_d$')
plt.ylabel(r'$y/L_d$')
plt.colorbar()
plt.title('Layer 2 PV')

plt.subplot(133)
plt.pcolormesh(m.x/m.rd,m.y/m.rd,(m.q[m.nz-1,]+m.Qy[m.nz-1]*m.y),cmap='Spectral_r')
plt.xlabel(r'$x/L_d$')
plt.ylabel(r'$y/L_d$')
plt.colorbar()
plt.title('Layer 3 PV')

## PCA

In [ ]:
pca = PCA(n_components=5)
pca.fit(m.v.reshape(10,-1).T)
pca.explained_variance_ratio_

In [ ]:
plt.figure(figsize=(10,6))

plt.subplot(121)
plt.plot(pca.components_.T,z_layer[:-1])
plt.plot(pca.components_[0,:],z_layer[:-1], 'k',linewidth=3,
         label='{:0.2f}%'.format(pca.explained_variance_ratio_[0]*100))
plt.xlabel(r'PCA amplitude')
plt.ylabel(r'depth (m)')
plt.legend()

plt.subplot(122)
plt.plot(m.pmodes[:,1:4],z_layer[:-1])
plt.plot(m.pmodes[:,1],z_layer[:-1], 'k',linewidth=3)
plt.xlabel(r'mode amplitude')
plt.ylabel(r'depth (m)')

plt.savefig("/Users/cwortham/Documents/research/rough_topo/figures/PCA.pdf", bbox_inches='tight')

## workspace

In [ ]:
m.pmodes.shape

In [ ]:
plt.figure()
plt.pcolormesh(m.y[:,0],z_layer[:-1],m.q[:,:,3],cmap='Spectral_r')
plt.colorbar()

In [ ]:
plt.figure()
plt.pcolormesh(m.y[:,0],z_layer[:-1],m.v[:,1,:],cmap='Spectral_r')
plt.colorbar()

In [ ]:
pca = PCA(n_components=4)
pca.fit(m.q.reshape(10,-1).T)

In [ ]:
pca.components_.shape

In [ ]:
pca.explained_variance_ratio_

In [ ]:
plt.figure()
plt.plot(pca.components_.T,z_layer[:-1])

In [ ]:
plt.figure()
plt.plot(m.pmodes[:,1:4],z_layer[:-1])

In [ ]:
H